In [1]:
from Okezone import Okezone
from datetime import timedelta, date, datetime
from IPython.display import clear_output

def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

In [2]:
obj = Okezone()

In [11]:
end_date = date(2018, 8, 12)
start_date = date(2018, 2, 2)
init_page = 33

In [12]:
for single_date in daterange(start_date, end_date):
    clear_output()
    print(single_date.strftime("%Y-%m-%d"))
    obj.getAllBerita([], init_page, 0, single_date.strftime("%Y/%m/%d"))
    init_page = 1

2018-02-02
page  10
https://index.okezone.com/bydate/index/2018/02/02/0/
memasukkan berita id  1854077
Insert berita  Rahasia Awet Muda Rossa, Banyak-Banyak Tertawa
salah2
memasukkan berita id  1854080
Insert berita  Gadis Buta di Video Klip Terbaru Setia Band Bikin <i>Baper</i>
salah2
memasukkan berita id  1854075
Insert berita  Berani Lawan LGBT di Aceh, AKBP Untung Sangaji Tuai Pujian
salah2
memasukkan berita id  1854079
Insert berita  Demi Ibadah, Zaskia Gotik Tak Sabar Ingin Menikah
salah2
memasukkan berita id  1854012
Insert berita  1.356 Peserta Ikuti GFNY Bali 2018
salah2
memasukkan berita id  1854064
Insert berita  Dipecat Jadi Karyawan, Pria Ini Tekuni Bisnis Sabu
salah2
memasukkan berita id  1853887
Insert berita  Rahmad Darmawan Sebut Arema FC Miliki 2 Kelebihan
salah2
memasukkan berita id  1854078
Insert berita  Lewat Foto Jadul, Ussy Sulistiawaty Kenang Momen Saling Menguatkan
salah2
memasukkan berita id  1854071
Insert berita  Diduga Terpeleset, Mayat Pria Tanpa Identita

masuk
memasukkan berita id  1853805
Insert berita  Detik-Detik <i>Center</i> Phoenix Suns Alami Patah Kaki Mengerikan
masuk
memasukkan berita id  1853804
Insert berita  Ada Kepentingan di Balik Upaya Setnov Rahasiakan Aktor Besar Kasus E-KTP
masuk
memasukkan berita id  1853801
Insert berita  Pesan Bahan Baku Pembuatan Ekstasi dari Belanda, <i>Debt Collector</i> Ditangkap
masuk
memasukkan berita id  1853797
Insert berita  Menko Darmin: Gubernur BI Baru Harus Paham Sektor Riil
masuk
memasukkan berita id  1853798
Insert berita  Mobil yang Terbakar Tabrak Pejalan Kaki di Shanghai, 18 Orang Luka-Luka 
masuk
page  15
https://index.okezone.com/bydate/index/2018/02/02/210/
memasukkan berita id  1853733
Insert berita  Tuntaskan Masalah Komedo Membandel dengan 5 Racikan Masker Alami
masuk
memasukkan berita id  1853795
Insert berita    Polisi Periksa Kejiwaan Pelaku Penganiayaan Ustadz Prawoto   
masuk
memasukkan berita id  1853794
Insert berita  Diperiksa Polisi, Kadishub DKI Dicecar soal Izin A

masuk
memasukkan berita id  1853726
Insert berita  Tumpukan Batu Misterius di Tengah Sungai Gegerkan Warga Sukabumi
masuk
memasukkan berita id  1853725
Insert berita    Kapal Pembawa Sembako Terbakar, 1 ABK Hilang   
masuk
memasukkan berita id  1853724
Insert berita  2 Pemain Anyar Resmi Diperkenalkan Persib
masuk
memasukkan berita id  1853722
Insert berita  Masih Hidup, Korban Percobaan Bunuh Diri di Stasiun Cikini Dilarikan ke RSCM
masuk
memasukkan berita id  1853720
Insert berita  Harum Energy <i>Buyback</i> 24,15 Juta Saham Rp52,81 Miliar
masuk
memasukkan berita id  1853719
Insert berita  Polisi 'Incar' Pembeli dan Pengunggah Video Porno Bocah di Bandung   
masuk
memasukkan berita id  1853718
Insert berita  Sakit Hati, Annisa Bahar Tak Mau Anggap Juwita Bahar Anak   
masuk
memasukkan berita id  1853717
Insert berita  Pembangunan Pos Lintas Batas Negara Demi Harga Diri RI
masuk
memasukkan berita id  1853716
Insert berita  Man United Diterpa Badai Cedera Jelang Laga Kontra Huddersfie

masuk
memasukkan berita id  1853652
Insert berita  Guru Seni di Sampang Tewas Dianiaya Siswa, Diduga Tak Terima Ditegur
masuk
page  24
https://index.okezone.com/bydate/index/2018/02/02/345/
memasukkan berita id  1853367
Insert berita  USU Gelar Karya Ilmiah Nasional Terbesar Scripta Research Festival 2018
masuk
memasukkan berita id  1853651
Insert berita  Penting! 4 Aksesoris Ini Dilarang Dipakai di Mobil Anda
masuk
memasukkan berita id  1853650
Insert berita   Ternyata Anak Juga Bisa Posesif Lho, <i>Yuk</i> Kenali Tahapannya 
masuk
memasukkan berita id  1853649
Insert berita  Wilayah Pekanbaru Alami Pemadaman Listrik Selama 16 Jam
masuk
memasukkan berita id  1853648
Insert berita  Mudah Lelah Saat Haid? Ini 5 Tips Mengatasinya
masuk
memasukkan berita id  1853647
Insert berita  Mahasiswa Hadiahi Jokowi 'Kartu Kuning' saat Dies Natalis ke-68 UI
masuk
memasukkan berita id  1853645
Insert berita  Bank Dunia Ingatkan Investasi Sumber Daya Manusia
masuk
memasukkan berita id  1853644
Insert 

masuk
memasukkan berita id  1853536
Insert berita  Burung Kakaktua Ini Jadi Penumpang Gelap Kapal Pesiar
masuk
memasukkan berita id  1853595
Insert berita  Verifikasi Faktual Partai Peserta Pemilu 2019, Perindo Sumut: Kita Sudah 100 Persen
masuk
memasukkan berita id  1853589
Insert berita  Gerindra: Sopir Angkutan Lama Tinggal di DKI Perlu Dibantu Buat KTP 
masuk
memasukkan berita id  1853552
Insert berita  Cabor Lawn Bowls Tak Pasang Target Tinggi di Asian Para Games 2018
masuk
memasukkan berita id  1853594
Insert berita  Wall Street Ditutup <i>Mixed</i> Dipicu Laporan Keuangan Emiten
masuk
memasukkan berita id  1853585
Insert berita  Demi Tampil Sempurna, Shandy Aulia Rela Pakai Behel
masuk
page  29
https://index.okezone.com/bydate/index/2018/02/02/420/
memasukkan berita id  1853373
Insert berita  5 Langkah Sederhana Cegah Penipisan Rambut Menjelang Usia Senja
masuk
memasukkan berita id  1853580
Insert berita  Resmi Tambah Masa Bakti di Arsenal, Begini Komentar Ozil
masuk
memasukkan 

masuk
memasukkan berita id  1853529
Insert berita  Kenakan Behel, Shandy Aulia Akui Sempat Menangis Kesakitan
masuk
memasukkan berita id  1853265
Insert berita  5 Alasan Efek Minum Jus Bit seperti Viagra, Semakin Energik di Ranjang!
masuk
memasukkan berita id  1853543
Insert berita  Sambut May Day, Menaker Resmikan Liga Pekerja Indonesia
masuk
memasukkan berita id  1853542
Insert berita  Kebakaran di Cengkareng Tewaskan Satu Keluarga 
masuk
memasukkan berita id  1853482
Insert berita  Pemuda Asal Indonesia Menangi Kompetisi Desain Medali Youth Olympic Games 2018
masuk
memasukkan berita id  1853522
Insert berita  Ingin Bercinta Tapi Sedang Flu, Harus Bagaimana?
masuk
memasukkan berita id  1853532
Insert berita  Sekdes di Aceh Barat Daya Ancam Pukul Korbannya Sebelum Lakukan Sodomi
masuk
memasukkan berita id  1853458
Insert berita  Sensasi Pijat Yoni Bangkitkan Gairah Seksual Wanita hingga Orgasme
masuk
memasukkan berita id  1853468
Insert berita  David Silva Diragukan Tampil di Laga Man

ChunkedEncodingError: ('Connection broken: IncompleteRead(6119 bytes read, 4121 more expected)', IncompleteRead(6119 bytes read, 4121 more expected))